# Pregunta 2

A continuación se importan algunas librerías estándar de ``python`` que serán de utilidad. Luego se definen funciones auxiliares de teoría de números que serán usadas tanto para la implementación de generadores de números primos y co-primos aleatorios, como para el funcionamiento del protocolo **RSA**:

In [551]:
# Standard library
from math import ceil
from random import randint, randrange

# Auxiliary functions

# Fast exponentiation
def _fast_exp(a, b):
    if not b:
        return 1
    result = 1
    power = a
    while b:
        if b % 2:
            result = result * power
        b = b // 2
        power = power * power
    return result

# Fast modular exponentiation
def _fast_exp_mod(a, b, n):
    if not b:
        return 1
    if b:
        result = 1
        power = a
        while b:
            if b % 2:
                result = (power * result) % n
            b = b // 2
            power = (power * power) % n
        return result
    return _fast_exp_mod(_mod_inverse(a, n), -b, n)

# Modular inverse
def _mod_inverse(a, n):
    r, s, t = _ext_euclidean(a, n)
    return s % n

# Greatest common divisor
def _gcd(a, b):
    while b:
        temp = b
        b = a % b
        a = temp
    return a

# Extended Euclidean algorithm
def _ext_euclidean(a, b):
    r_0 = a
    s_0 = 1
    t_0 = 0
    r_1 = b
    s_1 = 0
    t_1 = 1
    while r_1:
        r_2 = r_0 % r_1
        s_2 = s_0 - (r_0 // r_1) * s_1
        t_2 = t_0 - (r_0 // r_1) * t_1
        r_0 = r_1
        s_0 = s_1
        t_0 = t_1
        r_1 = r_2
        s_1 = s_2
        t_1 = t_2
    return r_0, s_0, t_0

# Check if n is a power of another number (optimized considering n > 3)
def _is_power(n):
    k = 2
    lim = 4

    # In the first 10^5 primes, all factors smaller than 10^6 already appeared, so we focus on the larger ones
    while lim <= n and k < 52:  # 10^(6 * 52) > 10^309 = max_n
        if _has_integer_root(n, k):
            return True
        k = k + 1
        lim = lim * 2
    return False

# Check if n is a k-th degree power (optimized considering n > 3)
def _has_integer_root(n, k):
    a = 1
    while _fast_exp(a, k) < n:
        a = a * 2
    return _has_integer_root_interval(n, k, a // 2, a)

# Check if n is a k-th degree power inside an interval
def _has_integer_root_interval(n, k, i, j):
    while i <= j:
        if i == j:
            return n == _fast_exp(i, k)
        p = (i + j) // 2
        val = _fast_exp(p, k)
        if n == val:
            return True
        elif val < n:
            i = p + 1
        else:
            j = p - 1
    return False

# Primality test (Solovay – Strassen) with some slight optimizations
def _primality_test(n, k):
    if _is_power(n):
        return False
    neg = 0
    for _ in range(k):
        a = randint(2, n - 1)
        if _gcd(a, n) > 1:
            return False
        b = _fast_exp_mod(a, (n - 1) // 2, n)
        if b == n - 1:
            neg = neg + 1
        elif b != 1:
            return False
    return neg > 0

# Obtain all primes smaller or equal to n (Sieve of Eratosthenes)
def _get_primes(n):
    prime_idx = [1] * (n + 1)
    p = 2
    while (p * p) <= n:
        if prime_idx[p]:
            for i in range(p * p, n + 1, p):
                prime_idx[i] = 0
        p = p + 1
    return [p for p in range(2, n + 1) if prime_idx[p]]

# Generate a pair of unique random prime numbers in the interval [a,b]
def _generate_primes(a, b):
    # Number of repetitions for the primality test
    k = 34  # Probability of error is (1/2)^k

    # Trivial case
    if a == 2:
        return [2, 3]

    # Pre-calculate first 10^5 primes
    first_primes = _get_primes(100000)

    # Search for primes by discarding most candidates in constant time
    primes = []
    while True:
        p = randrange(a + 1, b + 1, 2)  # Only check odd numbers
        if p in primes:  # Both primes need to be different
            continue
        candidate = True
        for n in first_primes:  # Discard with first n prime factors
            if p <= n:
                break
            if not (p % n):
                candidate = False
                break
        if candidate and _primality_test(p, k):  # Test for primality
            primes.append(p)
            if len(primes) == 2:
                return primes

# Generate a random number that is co-prime to n
def _generate_coprime(n):
    # Trivial case
    if n == 2:
        return 1

    # Search for co-primes
    while True:
        c = randint(2, n - 1)
        if _gcd(c, n) == 1:  # GCD must be 1
            return c

Ahora se construyen las clases **RSAReceiver** y **RSASender**, que modelan el comportamiento del emisor y receptor de un mensaje, utilizando el cifrado asimétrico **RSA**. El receptor genera una llave pública y otra privada, entregándole la pública al emisor para que pueda enviarle un mensaje cifrado. Por otro lado, el emisor sólo necesita recibir la llave pública del receptor (sin necesidad de generar las propias), con lo que puede encriptar el mensaje y enviárselo al receptor, que es el único capaz de desencriptarlo al tener la llave privada:

In [552]:
# RSA

# Receiver
class RSAReceiver:
    def __init__(self, bit_len):
        """
        Arguments:
            bit_len: A lower bound for the number of bits of N,
            the second argument of the public and secret key.
        """
        # Generate random primes P,Q
        p, q = _generate_primes(2 ** ceil(bit_len / 2), 2 ** (ceil(bit_len / 2) + 1) - 1)

        # Define N and phi(N)
        n = p * q
        phi_n = n - p - q + 1

        # Generate random co-prime d (to phi(N))
        d = _generate_coprime(phi_n)

        # Get the modular inverse e for d in mod phi(N)
        e = _mod_inverse(d, phi_n)

        # Define private & public keys
        self.private_key = (d, n)
        self.public_key = (e, n)

    # Get public key
    def get_public_key(self):
        """
        Returns:
            public_key: Public key expressed as a Python 'bytearray' using the
            PEM format. This means the public key is divided in:
            (1) The number of bytes of e (4 bytes)
            (2) The number e (as many bytes as indicated in (1))
            (3) The number of bytes of N (4 bytes)
            (4) The number N (as many bytes as indicated in (3))
        """
        # Construct PEM key
        e_bytes = ceil(self.public_key[0].bit_length() / 8)
        pem_key = bytearray(e_bytes.to_bytes(4, 'big'))
        pem_key += self.public_key[0].to_bytes(e_bytes, 'big')
        n_bytes = ceil(self.public_key[1].bit_length() / 8)
        pem_key += bytearray(n_bytes.to_bytes(4, 'big'))
        pem_key += self.public_key[1].to_bytes(n_bytes, 'big')
        return pem_key

    # Decrypt the cipher
    def decrypt(self, ciphertext):
        """
        Arguments:
            ciphertext: The ciphertext to decrypt.
        Returns:
            message: The original message.
        """
        # Message to decrypt
        encoded = bytearray()

        # Block-based decryption
        len_block = ceil(self.private_key[1].bit_length() / 8)
        n_blocks = len(ciphertext) // len_block
        for block in range(n_blocks):
            start = block * len_block
            stop = start + len_block
            c = int.from_bytes(ciphertext[start: stop], 'big')
            m = _fast_exp_mod(c, self.private_key[0], self.private_key[1])
            encoded += m.to_bytes(ceil(m.bit_length() / 8), 'big')
        return encoded.decode('utf-8')

# Sender
class RSASender:
    def __init__(self, public_key):
        """
        Arguments:
            public_key: The public key that will be used to encrypt messages.
        """
        # Reconstruct (e,n) pair from PEM key
        start = 0
        stop = 4
        e_bytes = int.from_bytes(public_key[start: stop], 'big')
        start = stop
        stop += e_bytes
        e = int.from_bytes(public_key[start: stop], 'big')
        start = stop
        stop += 4
        n_bytes = int.from_bytes(public_key[start: stop], 'big')
        start = stop
        stop += n_bytes
        n = int.from_bytes(public_key[start: stop], 'big')
        self.receiver_key = (e, n)

    # Encrypt the message
    def encrypt(self, message):
        """
        Arguments:
            message: The plaintext message to encrypt.
        Returns:
            ciphertext: The encrypted message.
        """
        # Message encoded with utf-8
        encoded = bytearray(message, 'utf-8')
        ciphertext = bytearray()

        # Block-based encryption
        len_block = ceil(self.receiver_key[1].bit_length() / 8) - 1
        n_blocks = ceil(len(encoded) / len_block)
        for block in range(n_blocks):
            start = block * len_block
            stop = start + len_block
            if block == n_blocks - 1:  # Last block
                mod = len(encoded) % len_block
                if mod:
                    stop = start + mod
            m = int.from_bytes(encoded[start: stop], 'big')
            c = _fast_exp_mod(m, self.receiver_key[0], self.receiver_key[1])
            ciphertext += c.to_bytes(len_block + 1, 'big')
        return ciphertext

Se prueba el funcionamiento del protocolo **RSA** enviando mensajes de largo arbitrario entre el emisor y receptor:

In [553]:
# Test RSA protocol
if __name__ == '__main__':
    """
    receiver = RSAReceiver(2048)
    sender = RSASender(receiver.get_public_key())
    message = 'Gol D. Roger was known as the Pirate King, the strongest and most infamous being to have sailed the Grand Line. The capture and execution of Roger by the World Government brought a change throughout the world. His last words before his death revealed the existence of the greatest treasure in the world, One Piece. It was this revelation that brought about the Grand Age of Pirates, men who dreamed of finding One Piece-which promises an unlimited amount of riches and fame-and quite possibly the pinnacle of glory and the title of the Pirate King.'
    cipher = sender.encrypt(message)
    print(cipher.hex().upper())
    text = receiver.decrypt(cipher)
    print(text)
    """
    from base64 import b64encode, b64decode
    public_key = b64decode('AAAB/0/aZPUJZYIXWEi0pd8e7FWeVDRl5brYSS8H2wFIg8Q1H1AnM3uVHWB7Bi6j6E46124UNn1dpM85xJdtN9mFFzEAAAIBAY9E+CeZ0rzJa5I7FeAQL6+U8x3nU/YMdoLW69Tsv7kKnpMWhQQIdYHYDXTRiKM0qtdkKmaUFPptFFhTscFNZvE=')
    print(public_key)
    text = (
    'Being open source means anyone can independently review '
    'the code. If it was closed source, nobody could verify the '
    'security. I think it’s essential for a program of this '
    'nature to be open source.'
    )
    sender = RSASender(public_key)
    print(sender.receiver_key)
    #cipher = sender.encrypt(text)
    #print(b64encode(cipher))

b"\x00\x00\x01\xffO\xdad\xf5\te\x82\x17XH\xb4\xa5\xdf\x1e\xecU\x9eT4e\xe5\xba\xd8I/\x07\xdb\x01H\x83\xc45\x1fP'3{\x95\x1d`{\x06.\xa3\xe8N:\xd7n\x146}]\xa4\xcf9\xc4\x97m7\xd9\x85\x171\x00\x00\x02\x01\x01\x8fD\xf8'\x99\xd2\xbc\xc9k\x92;\x15\xe0\x10/\xaf\x94\xf3\x1d\xe7S\xf6\x0cv\x82\xd6\xeb\xd4\xec\xbf\xb9\n\x9e\x93\x16\x85\x04\x08u\x81\xd8\rt\xd1\x88\xa34\xaa\xd7d*f\x94\x14\xfam\x14XS\xb1\xc1Mf\xf1"
b''
(61654846105916176421358116234694990568501776751248565424387871109857144001720461260450204372351808963631572369280949868913976987196611881404734553220776953471093822045466517716356505149004317096734683041049778202857361300690482248507429769322663051508460117401173767305700912428344914108828296739567171867878672158254833, 0)
